In [1]:
import numpy as np
import pandas as pd 
import requests 
from bs4 import BeautifulSoup
import mysql.connector
from sqlalchemy import create_engine

In [2]:
db_name = "World_Economies.db"
table_name = "Country_by_GDP"
csv_path = "Country_by_GDP.csv"

In [3]:
url = "https://web.archive.org/web/20230902185326/https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29"

In [4]:
data = requests.get(url).text

In [5]:
soup = BeautifulSoup(data, "html.parser")

In [6]:
tables = soup.find_all("tbody")

In [7]:
gdp_table = tables[2]

In [9]:
table_attribs = ["Country", "GDP_USD_millions"]

In [30]:
def extract():
    country_by_GDP = pd.DataFrame(columns= table_attribs)
    url = "https://web.archive.org/web/20230902185326/https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29"
    data = requests.get(url)
    soup = BeautifulSoup(data.content)
    tables = soup.find_all("tbody")
    gdp_table = tables[2]
    
    for row in gdp_table.find_all("tr"):
        cols = row.find_all("td")
        if cols and "—" not in [col.text.strip() for col in cols[2]]:
            first_column = cols[0]
            links = first_column.find_all("a")
            if links:
                Country = cols[0].text.strip()
                GDP_USD_millions = cols[2].text.strip()
                country_by_GDP = pd.concat([country_by_GDP, pd.DataFrame([{"Country": Country, "GDP_USD_millions": GDP_USD_millions}])], axis= 0, join= "outer", ignore_index = True)
                                                   
    return country_by_GDP

In [31]:
extract()

,Country,GDP_USD_millions
0,United States,"26,854,599"
1,China,"19,373,586"
2,Japan,"4,409,738"
3,Germany,"4,308,854"
4,India,"3,736,882"
...,...,...
186,Marshall Islands,291
187,Palau,262
188,Kiribati,248
189,Nauru,151


In [ ]:
def extract():
    country_by_GDP = pd.DataFrame(columns= table_attribs)
    url = "https://web.archive.org/web/20230902185326/https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29"
    data = requests.get(url)
    soup = BeautifulSoup(data.content)
    tables = soup.find_all("tbody")
    gdp_table = tables[2]
    
    for row in gdp_table.find_all("tr"):
        cols = row.find_all("td")
        if len(cols) != 0:
            if cols[0].find("a") is not None and "—" not add
        